## Create a new

In [19]:
from pyautocad import APoint, Autocad

acad = Autocad(create_if_not_exists=True, visible=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing3.dwg


In [20]:
acad.app

In [21]:
acad.doc, acad.ActiveDocument

(<comtypes.client.lazybind.Dispatch at 0x22a1a738440>,
 <comtypes.client.lazybind.Dispatch at 0x22a1a712fc0>)

In [22]:
acad.model

#### iter_layouts
1. doc – document to iterate layouts from if doc=None (default), ActiveDocument is used
2. skip_model – don’t include ModelSpace if True

In [29]:
list(acad.iter_layouts(doc=None, skip_model=True))

#### iter_objects(object_name_or_list=None, block=None, limit=None, dont_cast=False)
1. object_name_or_list – part of object type name, or list of it
2. block – Autocad block, default - ActiveDocument.ActiveLayout.Block
3. limit – max number of objects to return, default infinite
4. dont_cast – don’t retrieve best interface for object, may speedup iteration. Returned objects should be casted by caller

In [37]:
list(acad.iter_objects(object_name_or_list=None, block=None, limit=None, dont_cast=False))

[<POINTER(IAcadCircle) ptr=0x22a193b5858 at 22a1cfefd50>,
 <POINTER(IAcadLine) ptr=0x22a193b52d8 at 22a1cfee6d0>,
 <POINTER(IAcadArc) ptr=0x22a193b5058 at 22a1cfec0d0>,
 <POINTER(IAcadLWPolyline) ptr=0x22a18faa1b8 at 22a1cfef750>,
 <POINTER(IAcadLWPolyline) ptr=0x22a17da47d8 at 22a1cfefe50>,
 <POINTER(IAcadMText) ptr=0x22a18f92778 at 22a1cfef1d0>]

#### iter_objects_fast(object_name_or_list=None, container=None, limit=None)
1. Shortcut for iter_objects(dont_cast=True)
2. Shouldn’t be used in normal situations

#### find_one(object_name_or_list, container=None, predicate=None)
Returns first occurance of object which match predicate

1. Parameters:	
1. object_name_or_list – like in iter_objects()
2. container – like in iter_objects()
3. predicate – callable, which accepts object as argument and returns True or False
2. Returns:	
1. Object if found, else None

In [48]:
def f(x):
    if x.center[0] < 10:
        return True

acad.find_one(object_name_or_list="Circle", container=None, predicate=f)

<POINTER(IAcadCircle) ptr=0x22a193b5858 at 22a1d0c35d0>

#### best_interface(obj)

In [53]:
p1 = APoint(0, 0)
c1 = acad.model.AddCircle(p1, 2)

In [54]:
acad.best_interface(c1)

<POINTER(IAcadCircle) ptr=0x22a18f98728 at 22a1d1442d0>

#### prompt(text)
1. Prints text in console and in AutoCAD prompt

In [55]:
acad.prompt("Hello World")

Hello World


#### get_selection(text='Select objects')
1. Asks user to select objects

2. Parameters:	text – prompt for selection

In [56]:
acad.get_selection(text="select_object")

select_object


#### distance_to(other)
1. Parameters:	other – APoint instance or any sequence of 3 coordinates

#### pyautocad.types.distance(p1, p2)[source]
Returns distance between two points p1 and p2

## Drawin Basic Shapes
#### Lines, Circles, Rectangles, Text

In [60]:
p1 = APoint(0, 0)
p2 = APoint(50, 25)
for i in range(5):
    text = acad.model.AddText("hello world", p1, 2.5)
    acad.model.AddLine(p1, p2)
    acad.model.AddCircle(p1, 10)
    p1.y += 10

### Working with existing objects

##### Iterate over objects

In [49]:
for obj in acad.iter_objects():
    print(obj.ObjectName)

AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle
AcDbText
AcDbLine
AcDbCircle


In [50]:
for obj in acad.iter_objects("Text"): # or tex
    print(obj.TextString, obj.InsertionPoint)

for obj in acad.iter_objects(["Text", "Line"]):
    print(obj.ObjectName)

hello world (0.0, 0.0, 0.0)
hello world (0.0, 10.0, 0.0)
hello world (0.0, 20.0, 0.0)
hello world (0.0, 30.0, 0.0)
hello world (0.0, 40.0, 0.0)
AcDbText
AcDbLine
AcDbText
AcDbLine
AcDbText
AcDbLine
AcDbText
AcDbLine
AcDbText
AcDbLine


In [51]:
acad.model.AddText("hello_world_3", p1, 2.5)

In [52]:
def text_contains_3(text_object):
    result = "3" in text_object.TextString
    return result

text = acad.find_one("Text", predicate=text_contains_3)
print(text.TextString)

hello_world_3


In [53]:
from pyautocad import ACAD

for text in acad.iter_objects('Text'):
    old_insertion_point = APoint(text.InsertionPoint)
    text.Alignment = ACAD.acAlignmentRight
    text.TextAlignmentPoint = old_insertion_point

## Tables

In [55]:
from pyautocad import Autocad, APoint
from pyautocad.contrib.tables import Table

acad = Autocad()
p1 = APoint(0, 0)
for i in range(5):
    obj = acad.model.AddText(u'Hi %s!' % i, p1, 2.5)
    p1.y += 10

In [58]:
table = Table()
for obj in acad.iter_objects('Text'):
    x, y, z = obj.InsertionPoint
    table.writerow([obj.TextString, x, y, z])
table.save('data.xls', 'xls')

AttributeError: 'Dataset' object has no attribute 'xls'